figure = 66.66
bottle = 62.96
pen = 54.54

In [1]:
import sys
sys.path.append(r"C:\Users\49174\anaconda3\envs\praktikum\Lib\site-packages")
import torch
import numpy as np
from dataset_jacquard_samples import JacquardSamples
from utils import get_transform, get_inv_transform
from bce_model import BCEGraspTransformer
from utils_train import create_correct_false_points
import random
from pytorch_lightning.loggers import TensorBoardLogger
import matplotlib.pyplot as plt
from metrics_utils import * 
from infer_utils import * 

xFormers not available
xFormers not available


In [2]:
IMAGE_SIZE = 1120 
PATCH_DIM = IMAGE_SIZE //14
IDX = 3 #id of the object in the category

In [3]:
split_train = r"Objects_train/"
split_test = r"Objects_test/"

## Init the Valid Input Point Model 

In [4]:
args_infer = {
    "model_path" : "runs/bottle_2_single.ckpt",
    "device" : "cuda",
    "img_size" : IMAGE_SIZE
}

In [5]:
device = torch.device(args_infer["device"]) if torch.cuda.is_available() else torch.device("cpu")
image_transform = get_transform()
inv_transform = get_inv_transform()
model = BCEGraspTransformer(img_size=IMAGE_SIZE,int_dim=256,output_dim=128,input_cls=1)
dataset_train = JacquardSamples(dataset_root= split_train ,image_transform=image_transform, num_targets=5, overfit=False,
                              img_size=args_infer["img_size"], idx=4)
dataset = JacquardSamples(dataset_root= split_test ,image_transform=image_transform, num_targets=5, overfit=False,
                              img_size=args_infer["img_size"], idx=7)
model.load_state_dict(torch.load(args_infer["model_path"]))
model = model.to(device)

classes ['8b5a96b72767f7354fac5eaf08c4d9ce', 'acb1617efd51854339ad6b48a26028ff', '82a0ba979f437d0730d32956603b1f19', 'd67f1effed6e65fb2c45f75322191dd7', '7c6491c7fe65449cd9bdef3310c0e5a6', '611f9690f6eb4da79a6a8a8f3fa61f11', '6f67cd916a3cb630853d6ffb3e180267', '4ca3850427226dd18d4d8cbfa622f635']
classes ['1cbc0504316634ed80f698ab340a8570', '334bf3542211da33e858db1dc3499392', '2497bd27b94a197b698580924e57a755', 'abc933b264e3282442a65392abbe6885', '4c992058eca403d9980a39e0273687b1', 'edcbeac5da0b42dd7936acbca381136f', 'b1f4459eae651d4a1e4a40c5ca6a557d', 'e7b9fef210fa47505615f13c9cba61e5', 'e6809c021f1319b035836c728d324152', '351f967473e61599ca5852e989a25dd3', '812f1cbea6093044e3f7a74e12a274ef', 'ffc8c740dfb066de2e5e4d317752daae', 'bb66f1e88c53601726f0038a269bcb7c', '5f5271bf1d62599de3f7a74e12a274ef', 'd74bc917899133e080c257afea181fa2', '8bf17934792a107ee3f7a74e12a274ef', '866ba2af4ee624d0f44c04861b156235', 'b095a1c41447e9da887d2d6e4a6617b3', 'b0d444c3570b831be1e09e3c89252abb', '4e371e30a

## Init the Second Point UNet Model 

In [6]:
from unet import UNet

In [7]:
unet = UNet(n_channels=2,n_classes=1)
unet = unet.cuda()
unet.eval()

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

In [8]:
constrain_mode = True
TOPK = True
topk_grasps = 1
topk_input = 50

In [9]:
max_dist =0
min_dist = 999999
for i in range(len(dataset_train)):
    data = dataset_train[i]
    img = data["img"].to(device)
    img = torch.permute(img, (0, 2, 1))
    mask = data["mask"].sum().sqrt()
    all_points = torch.tensor(data["points_grasp"]//14)
    dif = (all_points[:, 0, :] - all_points[:, 1, :]).type(torch.float32).norm(p=2, dim=1)
    dif_n = (dif/mask).unsqueeze(1)
    if max_dist < dif_n.max():
        max_dist = dif_n.max()
    if min_dist > dif_n.min() and dif_n.min()>0:
        min_dist = dif_n.min()
if min_dist < 0.01:
    min_dist = 0.01

/home/cedric/ZeroShot-Grasping/MLP-approach/dataset_jacquard_samples.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_dict['points_grasp'] = torch.tensor(points_grasps)
/home/cedric/ZeroShot-Grasping/MLP-approach/dataset_jacquard_samples.py:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_dict['raw'] = torch.tensor(gknet_labels)
/tmp/ipykernel_22686/3396165159.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_points = torch.tensor(data["points_grasp"]//14)


### Loop over the data and get the metrics

In [10]:
import time
from tabulate import tabulate

In [11]:
%load_ext autoreload
accuracies = []
print_every = 100
for test_idx in range(len(dataset)): 
    #test_idx = 13
    start = time.time()
    data = dataset[test_idx]
    img = data["img"].to(device)
    mask = data["mask"].to(device)
    resized_mask = data['resized_mask'].to(device)
    img = torch.permute(img, (0, 2, 1))
    grasp = data["points_grasp"]//14
    normal_grasp = grasp.clone()
    grasp_inv = torch.cat([grasp[:,1,:].unsqueeze(1), grasp[:,0,:].unsqueeze(1)], dim=1)
    grasp = torch.cat([grasp, grasp_inv], dim=0)
    #get features from dino
    features,_ = model.forward_dino_features(img.unsqueeze(0))
    features = features.squeeze().reshape(PATCH_DIM, PATCH_DIM, 768)

    #plt.figure(figsize=(16,16))
    org_image = torch.permute(inv_transform(img), (1, 2, 0)).cpu().numpy()
    end = time.time() - start
    #plt.imshow(org_image)


    ## get valid points
    all_points = torch.tensor([[i,j] for j in range(PATCH_DIM) for i in range(PATCH_DIM)]).unsqueeze(1).to(device)

    start_valid_pts = time.time()
    preds, preds_patches = get_valid_points(all_points, features, model, device,PATCH_DIM=PATCH_DIM)
    end_valid_pts = time.time() - start_valid_pts 
    
    start_topk = time.time()
    preds_cp, pts, valid_pts_pred = get_topk_valid_points(preds,preds_patches, mask, topk_num=topk_input,TOP_K=TOPK)
    end_topk = time.time() - start_topk
    
    #get second point information using the unet inference 
    start_second_point = time.time()
    unet_preds, conf_vals = get_unet_preds(unet,valid_pts_pred,resized_mask)
    #need to add it to items array here 
    items = []
    #import pdb; pdb.set_trace()
    for i in range(len(unet_preds)):
        item = {}
        item['origin_point'] = valid_pts_pred[i]
        item['preds'] = preds
        item['single_point'] = valid_pts_pred[i]
        item['pred_point'] = unet_preds[i]
        item['conf'] = conf_vals[i]
        items.append(item)
        #items.append([valid_pts_pred[i],unet_preds[i]])
    
    end_second_point = time.time() - start_second_point
    num_grasps = valid_pts_pred.shape[0]
    #objs = 1


    datas = {}
    datas['mask'] = mask
    datas['valid_pts_pred'] = valid_pts_pred
    datas['min_dist'] = min_dist
    datas['max_dist'] = max_dist
    datas['heights'] = data['height'] 
    datas['args_infer'] = args_infer
    datas['mask_n'] = data["mask"]
    datas['device'] = device
    datas['features'] = features
    #data['model_single'] = model_single

    ##currently only support constraint mode == false

        

    start_vis = time.time()
    cur_accuracy = vis_preds_with_metrics(num_grasps, items, org_image, grasp,datas['heights'], args_infer,preds_cp,topks=topk_grasps)
    end_vis = time.time() - start_vis
        
    accuracies.append(cur_accuracy) 
    #except : 
    
    '''
    all_runtime = float(end + end_valid_pts + end_topk + end_second_point + end_pred + end_vis) 
    data = [['Dinov2',round(end / all_runtime,4) * 100, round(end,4) * 1000], 
            ['Valid Points', round(end_valid_pts/ all_runtime,2) * 100, round(end_valid_pts,4) * 1000],    
            ['Topk', round(end_topk/ all_runtime,4) * 100, round(end_topk,4) * 1000],
            ['Second Point', round(end_second_point/ all_runtime,4) * 100, round(end_second_point,4) * 1000],
            ['Prediction', round(end_pred/ all_runtime,4) * 100,round(end_pred,4) * 1000],
            ['Visualization',round( end_vis/ all_runtime,4) * 100,round(end_vis,4) * 1000],       
    ]
    
    headers = ['Module Type', 'Runtime Percentage', 'Time in ms']
    table = tabulate(data, headers=headers, tablefmt="grid")
    print(table)
    '''
        
    #except : 
    #    print('issue with eval')
    #    #accuracies.append(0)
     
    #print("idx", test_idx)
    if test_idx % print_every == 0: 
        print("Iteration", test_idx)
        print('Total accuracy is', sum(accuracies)/len(accuracies))


#print('Total accuracy is', sum(accuracies)/len(accuracies))

RuntimeError: 1D tensors expected, but got 2D and 1D tensors

<Figure size 1600x1600 with 0 Axes>

In [ ]:
print(org_image.shape)

(1120, 1120, 3)


In [ ]:
print(torch.unique(preds))

tensor([0.0000e+00, 1.1426e-06, 1.2221e-06,  ..., 9.8018e-01, 9.8485e-01,
        9.8706e-01])


In [ ]:
plt.figure(figsize=(16,16))
plt.imshow(org_image + origin_point*0.7)

NameError: name 'origin_point' is not defined

<Figure size 1600x1600 with 0 Axes>